# Depths and Snow Pit Data Package Contents

In [ ]:
# import libraries / packages
import os
from pathlib import Path
import glob
import pandas as pd
import numpy as np

#plotting imports
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use(['seaborn-notebook'])


# Download snow depth data from NSIDC
From the SnowEx20 [Depth Probe Landing Page](https://nsidc.org/data/SNEX20_SD/versions/1), you can download data and access the [User's Guide](https://nsidc.org/sites/nsidc.org/files/SNEX20_SD-V001-UserGuide_1.pdf)
<img src="images/SD-landingPage.png">

The Community Snow Depth Probe data package is a single CSV with over 36,000 geolocated snow depths! Three different instrument types were used to measure depths and are recorded in the Measurement Tool column.

Instructions:
1) Click on link to landing page
1) Go to bottom of page
1) Click on download script
1) Save to your snowex21/scripts folder locally

## Download snow pit data from NSIDC
From the SnowEx20 [Snow Pit Landing Page](https://nsidc.org/data/SNEX20_GM_SP/versions/1), you can download data and access the [User's Guide](https://nsidc.org/data/SNEX20_GM_SP/versions/1). 

<img src="images/SP-landingPage.png">

## Programmatically download snow pit data from NSIDC

In [ ]:
# load snow pit data
%run '../scripts/nsidc-download_SNEX20_GM_SP.001_2021-07-15.py'
print('Grand Mesa 2020 Snow Pit data download complete')

In [ ]:
%%bash

# do some file organizing by moving those files to a new directory
mv *.csv ../data/pits/
mv *.xml ../data/pits/
mv *.jpg ../data/pits/
mv *.pdf ../data/pits/

In [ ]:
# show filename in new location:
path = Path('../data/pits/')

for i, filename in enumerate(path.glob('*1N6*')):
    print(i, filename.name)

### Don't want to work with all the files? Method to filter files

In [ ]:
# what files would you like to find?
parameter = 'temperature'
pitID = '1N6'
date = '20200128'

### Read the Pit Parameter File

In [ ]:
# show filename in new location:
path = '../data/pits/'

filename = path+'SnowEx20_SnowPits_GMIOP_'+date+'_'+pitID+'_'+parameter+'_v01.csv'
print(filename)

df = pd.read_csv(filename, header=7)

In [ ]:
# check data types for each column
df.dtypes

### Plotting

In [ ]:
# plot temperature
ax = df.plot(x='Temperature (deg C)',y='# Height (cm)', legend=False)
ax.set_aspect(0.4)
ax.grid()
ax.set_title('pit {}: Temperature Profile'.format(pitID))
ax.set_xlabel('Temperature (deg C)')
ax.set_ylabel('Snow Depth (cm)')